In [1]:
# !pip uninstall -y flash-attn
# !pip install flash-attn --no-build-isolation

In [2]:
# !pip install --force-reinstall torch --index-url https://download.pytorch.org/whl/cu121

🔹 Why chrF2++ is preferred for EN→Indic MT

Indic languages (Marathi, Hindi, Bengali, Tamil, etc.) are:

morphologically rich

often longer than English

have free word order

contain agglutination and fused morphemes

Because of this:

chrF++ (β=3) tends to inflate scores for long/verbose translations

chrF2++ (β=2) balances precision and recall better

chrF2++ correlates more strongly with human judgment in EN→Indic evaluations
(reported in FLORES, WAT, ITTB, AI4Bharat, Microsoft MT work)

In [3]:
combined_punct_model = "thenlpresearcher/shalaka_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva"
combined_punct_model_data_changed = "thenlpresearcher/shalaka_fd_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva"
combined_punct_model_name = "thenlpresearcher/iitb_en_indic_robust_punctuation_model"

In [4]:
from datasets import load_dataset
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["sent_written"]
ref_gt     = raw_datasets['test']["gt_marathi"]
ref_gem    = raw_datasets['test']["gemini_out"]
ref_cfilt  = raw_datasets['test']["cfilt_out"]

In [6]:
import os
os.getcwd()

'/workspace/Approach2/shalaka'

In [7]:
!nvidia-smi

Mon Nov 24 18:02:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   56C    P0              71W / 300W |  24249MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### Original

In [8]:
model_name = "ai4bharat/indictrans2-en-indic-dist-200M"

In [9]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:3")

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

2025-11-24 18:02:54.940247: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-24 18:02:55.016544: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-24 18:02:57.458444: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: उत्सवी गायकांनी प्रार्थनेचा उच्चार केल्याने गायकवृंदाचा जप केल्याने ध्वनीमुद्रण वाढले...........................................................................................................................................................................................................................................
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दर्शविणाऱ्या सहा महिन्यांच्या वासराला तपासणीसाठी सादर करण्यात आले....................................................................................................................................................................................................................................
eng_Latn: Planning authorities should provide alternative locations for smal

In [10]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सवी गायकांनी प्रार्थनेचा उच्चार केल्याने गायकवृंदाचा जप केल्याने ध्वनीमुद्रण वाढले.


In [11]:
mode = "original"

In [12]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to shalaka_original_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 68.35, chrF++: 84.70, chrF2++: 82.58
GT Marathi → BLEU: 56.64
Gemini    → BLEU: 21.72
CFILT     → BLEU: 50.55

🎯 BEST REFERENCE = GT (by highest BLEU)
Metrics written to shalaka_punct_original_baseline_outputs_eval_metrics.txt


## Only Punctuations

In [13]:
model_name = "thenlpresearcher/iitb-en-indic-only-punct"

In [14]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:3")

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: mar_Deva eng_Latn उत्सव साजरा करणार\u093C्याने प्रार्थना केली तेव्हा गायकवृंदाचा जप केल्याने आवाज वाढला.
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: mar_Deva eng_Latn जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दिसून येत असलेल्या सहा महिन्यांच्या वासूला तपासणीसाठी सादर करण्यात आले.
eng_Latn: Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
mar_Deva: mar_Deva eng_Latn नियोजन प्राधिकरणांनी निवासी क्षेत्रात आक्षेपार्ह असलेल्या किंवा असणार\u093C्या लहान व्यवसायांसाठी पर्यायी ठिकाणे पुरवली पाहिजेत.
eng_Latn: As the machine develops the forms we use to record data from past projects will be amended.
mar_Deva: mar_Deva eng_Latn जसजसे मशीन विकसित होईल तसतसे आम्ही मागील प्रकल्पांमधील डेटा रेकॉर

In [15]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थना केली तेव्हा गायकवृंदाचा जप केल्याने आवाज वाढला.


In [16]:
mode = "only_punct"

In [17]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to shalaka_only_punct_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 60.76, chrF++: 80.54, chrF2++: 77.72
GT Marathi → BLEU: 42.34
Gemini    → BLEU: 21.21
CFILT     → BLEU: 49.06

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to shalaka_punct_only_punct_baseline_outputs_eval_metrics.txt


## Without Punctuations

In [18]:
model_name = "thenlpresearcher/iitb-en-indic-without-punct"

In [19]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:3")

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: mar_Deva eng_Latn उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार करताच गायकवृंदाच्या पठणाने आवाज वाढवला.
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: mar_Deva eng_Latn जन्मानंतर लगेचच अस्तित्वात असलेल्या सर्व चारही पायांमध्ये लंगडेपणा दर्शविणारी सहा महिन्यांची एक वासर तपासणीसाठी सादर करण्यात आली.
eng_Latn: Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
mar_Deva: mar_Deva eng_Latn नियोजन प्राधिकरणांनी छोट्या व्यवसायांसाठी पर्यायी ठिकाणे पुरवली पाहिजेत जी निवासी क्षेत्रात आक्षेपार्ह आहेत किंवा असतील.
eng_Latn: As the machine develops the forms we use to record data from past projects will be amended.
mar_Deva: mar_Deva eng_Latn जसजसे मशीन विकसित होते, तसतसे आपण मागील प्रकल्पांमधील डेटा रेकॉर्ड करण्यासाठ

In [20]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार करताच गायकवृंदाच्या पठणाने आवाज वाढवला.


In [21]:
mode = "without_punct"

In [22]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to shalaka_without_punct_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 63.10, chrF++: 80.97, chrF2++: 78.39
GT Marathi → BLEU: 43.28
Gemini    → BLEU: 24.66
CFILT     → BLEU: 51.86

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to shalaka_punct_without_punct_baseline_outputs_eval_metrics.txt


## Combined Model

In [23]:
model_name = combined_punct_model_name

In [24]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:3")

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: mar_Deva eng_Latn उत्सव साजरा करणार\u093C्याने प्रार्थनेचा सूर वाजवल्यामुळे गायकवृंदाचा जप केल्याने आवाज वाढला.
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: mar_Deva eng_Latn जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दर्शविणारा सहा महिन्यांचा एक वासर तपासणीसाठी सादर करण्यात आला.
eng_Latn: Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
mar_Deva: mar_Deva eng_Latn नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी पर्यायी ठिकाणे प्रदान केली पाहिजेत जी निवासी क्षेत्रात आक्षेपार्ह आहेत किंवा असतील.
eng_Latn: As the machine develops the forms we use to record data from past projects will be amended.
mar_Deva: mar_Deva eng_Latn जसजसे मशीन विकसित होईल तसतसे आपण मागील प्रकल्पांमधील डेटा रेकॉर्ड

In [25]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थनेचा सूर वाजवल्यामुळे गायकवृंदाचा जप केल्याने आवाज वाढला.


In [26]:
mode = "combined"

In [27]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_indictrans2_eval_metrics.txt")

✔ Saved predictions to shalaka_combined_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 64.73, chrF++: 81.78, chrF2++: 79.09
GT Marathi → BLEU: 43.30
Gemini    → BLEU: 25.18
CFILT     → BLEU: 54.13

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to shalaka_punct_combined_indictrans2_eval_metrics.txt


## Combined Model - Shalaka LR=5e-6 & Epochs=3 Changed

In [28]:
model_name = combined_punct_model

In [29]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:3")

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: mar_Deva eng_Latn उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार केल्यामुळे गायकवृंदाने जप केल्याने आवाज वाढला.
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: mar_Deva eng_Latn जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दर्शविणारा सहा महिन्यांचा वासर तपासणीसाठी सादर करण्यात आला.
eng_Latn: Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
mar_Deva: mar_Deva eng_Latn नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी पर्यायी ठिकाणे प्रदान केली पाहिजेत जी निवासी क्षेत्रात आक्षेपार्ह आहेत किंवा असतील.
eng_Latn: As the machine develops the forms we use to record data from past projects will be amended.
mar_Deva: mar_Deva eng_Latn जसजसे मशीन विकसित होईल तसतसे मागील प्रकल्पांमधील डेटा रेकॉर्ड करण्य

In [30]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार केल्यामुळे गायकवृंदाने जप केल्याने आवाज वाढला.


In [31]:
mode = "combined_le"

In [32]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_indictrans2_eval_metrics.txt")

✔ Saved predictions to shalaka_combined_le_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 70.01, chrF++: 85.50, chrF2++: 83.17
GT Marathi → BLEU: 49.12
Gemini    → BLEU: 24.27
CFILT     → BLEU: 58.46

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to shalaka_punct_combined_le_indictrans2_eval_metrics.txt


## Combined Model- Shalaka LR=5e-6 & Epochs=3 & Dataset Changed

In [33]:
model_name = combined_punct_model_data_changed

In [34]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:3")

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: mar_Deva eng_Latn उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार केल्यामुळे गायकवृंदाने जप केल्याने आवाज वाढला.
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: mar_Deva eng_Latn जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दर्शविणारा सहा महिन्यांचा वासर तपासणीसाठी सादर करण्यात आला.
eng_Latn: Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
mar_Deva: mar_Deva eng_Latn नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी पर्यायी ठिकाणे पुरवली पाहिजेत जी निवासी क्षेत्रात आक्षेपार्ह आहेत किंवा असतील.
eng_Latn: As the machine develops the forms we use to record data from past projects will be amended.
mar_Deva: mar_Deva eng_Latn जसजसे मशीन विकसित होईल तसतसे मागील प्रकल्पांमधील डेटा रेकॉर्ड करण्यासाठी

In [35]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार केल्यामुळे गायकवृंदाने जप केल्याने आवाज वाढला.


In [36]:
mode = "combined_led"

In [37]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_indictrans2_eval_metrics.txt")

✔ Saved predictions to shalaka_combined_led_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 68.89, chrF++: 84.37, chrF2++: 82.12
GT Marathi → BLEU: 47.75
Gemini    → BLEU: 24.28
CFILT     → BLEU: 58.70

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to shalaka_punct_combined_led_indictrans2_eval_metrics.txt


## Combined Model - Shalaka LR=1e-5 & Epochs=5 Changed

In [38]:
model_name = combined_punct_model2

NameError: name 'combined_punct_model2' is not defined

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:3")

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


In [ ]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

In [ ]:
mode = "combined_le2"

In [ ]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f},  chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_indictrans2_eval_metrics.txt")

## Combined Model- Shalaka LR=1e-5 & Epochs=5 & Dataset Changed

In [ ]:
model_name = combined_punct_model_data_changed2

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:3")

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


In [ ]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

In [ ]:
mode = "combined_led2"

In [ ]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++ {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++ {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_indictrans2_eval_metrics.txt")